In [1]:
import mfem.ser as mfem
from glvis import glvis, to_stream

In [2]:
mesh = mfem.Mesh(5, 5, "TRIANGLE")
mesh.UniformRefinement()

# Create H1 finite element function space
fec = mfem.H1_FECollection(2, mesh.Dimension()) # order=2
fespace = mfem.FiniteElementSpace(mesh, fec)      

# Determine essential degrees of freedom (the whole boundary here)
ess_tdof_list = mfem.intArray()
ess_bdr = mfem.intArray([1]*mesh.bdr_attributes.Size())
fespace.GetEssentialTrueDofs(ess_bdr, ess_tdof_list)

# Define Bilinear and Linear forms for the Laplace problem -Δu=1
one = mfem.ConstantCoefficient(1.0)
a = mfem.BilinearForm(fespace)
a.AddDomainIntegrator(mfem.DiffusionIntegrator(one))
a.Assemble()
b = mfem.LinearForm(fespace)
b.AddDomainIntegrator(mfem.DomainLFIntegrator(one))
b.Assemble()

# Create a grid function for the solution and initialize with 0
x = mfem.GridFunction(fespace)
x.Assign(0.0)

# Form the linear system, AX=B, for the FEM discretization
A = mfem.OperatorPtr()
B = mfem.Vector()
X = mfem.Vector()
a.FormLinearSystem(ess_tdof_list, x, b, A, X, B)
print("Size of the linear system: " + str(A.Height()))

# Solve the system using PCG solver and get the solution in x
Asm = mfem.OperatorHandle2SparseMatrix(A)
Msm = mfem.GSSmoother(Asm)
mfem.PCG(Asm, Msm, B, X, 1, 200, 1e-12, 0.0)
a.RecoverFEMSolution(X, b, x)

Size of the linear system: 441


In [3]:
glvis((mesh, x), 400, 400)

glvis(data_str='MFEM mesh v1.0\n\n#\n# MFEM Geometry Types (see mesh/geom.hpp):\n#\n# POINT       = 0\n# SEGME…

In [4]:
g = glvis(to_stream(mesh,x) + 'keys ARjlmcbp*******')
g.set_size(600, 400)
g

glvis(data_str='MFEM mesh v1.0\n\n#\n# MFEM Geometry Types (see mesh/geom.hpp):\n#\n# POINT       = 0\n# SEGME…

In [1]:
stream = !curl -s https://glvis.org/data/streams/ex9.saved
stream = "\n".join(stream)

In [2]:
from glvis import glvis
glvis(stream)

glvis(data_str='MFEM mesh v1.0\n\n#\n# MFEM Geometry Types (see mesh/geom.hpp):\n#\n# POINT       = 0\n# SEGME…

In [10]:
print()

<function python_version at 0x7f364a0ae4c0>
